# import libraraies

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator #agumentation of pototo ,rotate , shift, brightness, prescale for normalization [0,1]range ,....
import numpy as np
import seaborn as sns
#from keras_preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import img_to_array,load_img
import matplotlib.pyplot as plt
import os
import splitfolders

# upload dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

* balanced dataset

In [ ]:
from zipfile import ZipFile
File_name='/content/drive/MyDrive/2500 Aug.zip'
with ZipFile(File_name,'r') as zip:
   zip.extractall()
   print('done')

In [ ]:
pic_size = 48    #48*48 input size

#split data into train and test and validatation

In [ ]:
path = "/content/images/"
!pip install split-folders

splitfolders.ratio(path, output="/content/new/", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [ ]:
train_dir = '/content/new/train/'
test_dir = '/content/new/test/'
val_dir='/content/new/val/'
base_path='/content/new/'
row = 48
col = 48
nb_classes = len(os.listdir('/content/new/train/'))
nb_classes

In [ ]:
plt.figure(0, figsize=(12,20))
cpt = 0

In [ ]:
for expression in os.listdir(base_path + "train/"):
    for i in range(1,6):
        cpt = cpt + 1
        plt.subplot(7,5,cpt)
        img = load_img(base_path + "train/" + expression + "/" +os.listdir(base_path + "train/" + expression)[i], target_size=(pic_size, pic_size))
        plt.imshow(img, cmap="gray")

plt.tight_layout()
plt.show()

* balanced dataset

In [ ]:
for expression in os.listdir(base_path + "train"):
    print(str(len(os.listdir(base_path + "train/" + expression))) + " " + expression + " images")

In [ ]:
labels = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:

sum = 0
for val in labels:
    count = len(os.listdir(base_path + "train/" + val))
    sum = sum + count
    print(val,count)
    plt.bar(val,count)
    plt.xlabel('Emotions')
    plt.ylabel('Number of Images')
    plt.title('Distribution of images of training dataset')
train_size = sum

In [ ]:
batch_size = 128

In [ ]:
#make agumentation for train and validation data
datagen_train = ImageDataGenerator(
    rescale=1.0 / 255.0,            # Normalize pixel values to [0, 1]
    rotation_range=20,              # Randomly rotate images in the range (degrees)
    zoom_range=0.2,                 # Randomly zoom in on images
                                   )
datagen_validation = ImageDataGenerator(
       rescale=1.0 / 255.0,            # Normalize pixel values to [0, 1]
    rotation_range=20,              # Randomly rotate images in the range (degrees)
    zoom_range=0.2,                 # Randomly zoom in on images
)
datagen_test=ImageDataGenerator(
       rescale=1.0 / 255.0,            # Normalize pixel values to [0, 1]
    rotation_range=20,              # Randomly rotate images in the range (degrees)
    zoom_range=0.2,                 # Randomly zoom in on images
)

In [ ]:
train_generator = datagen_train.flow_from_directory(base_path + "train",target_size=(pic_size,pic_size),color_mode="grayscale"
,batch_size=batch_size,class_mode='categorical',shuffle=True)
#train.flow: path, target_size :reshape all photos to the same size (48*48), color_mode:grayscale white and black ,batch_size=128
#class_mode=more than 2 classes make one hotencoding, shuffle=random the data not sequential
validation_generator = datagen_validation.flow_from_directory(base_path + "val",target_size=(pic_size,pic_size),color_mode="grayscale"
,batch_size=batch_size,class_mode='categorical',shuffle=False)
#same for validation data
test_generator = datagen_test.flow_from_directory(base_path + "test",target_size=(pic_size,pic_size),color_mode="grayscale"
,batch_size=batch_size,class_mode='categorical',shuffle=False)

In [ ]:
x_train, y_train = next(train_generator)
print(x_train.shape, y_train.shape)#check the reshape of the photos

# build model

In [ ]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam #optimizer as gradient descent take steps to minimium error
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
nb_classes = 7#outputs 7 experssions

In [ ]:
# Initialising the CNN
model = Sequential()

In [ ]:
#padding filter used 4 filters # 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))# 64 filter its size 3*3 ,shape of the image 48*48 ,1 for white and black colors ,padding os applied
model.add(BatchNormalization())
model.add(Activation('relu'))# activation layer using relu activation function
model.add(MaxPooling2D(pool_size=(2, 2)))#Max pooling is then used to reduce the spatial dimensions of the output volume. reduce size of photo to half
model.add(Dropout(0.25))#25% dropout of nodes randomly

In [ ]:
#2 - Convolution
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
# 3 - Convolution
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
# 4 - Convolution
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Flatten())#convert fron 2D to 1D


In [ ]:
#first hidden layer
model.add(Dense(256))#256 node in the first hidden layer
model.add(BatchNormalization())
model.add(Activation('relu'))#activation layer in the
model.add(Dropout(0.25))# dropout 25% of nodes randomly form first layer

In [ ]:
#second hidden layer
model.add(Dense(512))#512 node for 2nd layer
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

In [ ]:
model.summary()#depth increase and size of photos decreases

In [ ]:
#output layer
model.add(Dense(nb_classes, activation='softmax'))#activation function softmax


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
%%time
epochs = 50 #50 train step



In [ ]:


learn = ReduceLROnPlateau(monitor='val_loss',
                          patience=5,
                          verbose=1,
                          factor=0.2,
                          min_lr=0.00001)

early = EarlyStopping(monitor="val_loss",patience=3)

* save the model

In [ ]:


checkpointer = ModelCheckpoint(filepath='XRAY.keras', verbose=1, save_best_only=True)
callbacks_list = [checkpointer]


* train the model

In [ ]:
%%time

# number of epochs to train the NN
epochs = 50
history = model.fit(train_generator,
                                steps_per_epoch=train_generator.n//train_generator.batch_size,
                                epochs=epochs,
                                validation_data = validation_generator,
                                validation_steps = validation_generator.n//validation_generator.batch_size,
                                callbacks=callbacks_list
                                )
#fit model to train data and validation by taking 128 batches (batch_size)
#calculate acc and acc.loss, validation,valid.loss

In [ ]:
loss, accuracy = model.evaluate(test_generator)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

# evaluation

In [ ]:
predictions = model.predict(test_generator)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = test_generator.classes
class_names = test_generator.class_indices.keys()

from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

report_dict = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report_dict)